TODO: 
- integrate colab fold
- PIP freeze has to be added/check optimum setup
- run tests
- add option to provide mutations as a list
- Getting NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968 ; added work around to set UTF-8 encoding
- pka anii generates _prep.pdb files, do we need to use them?
- bash scripts will be finished successfully even if it went wrong. Need to check if file exists after it.

# <b><font color='#009e74'>End-to-end molecular dynamics simulation with AlphaFold and Gromacs </font></b>

Preprint pipeline version for running MD simulations using **AlphaFold and GROMACS**. The program, using as input a protein sequence returns MD trajectories and all the associated files to run it.
More details can be found in: **Manuscript...:** ["Manuscript title"](https://ref). Source code is available on the project [Github](https://github.com/ref) page.

##  <b><font color='#009e74'> Reminders and Important informations:</font></b>
- This notebook  <b><font color='#d55c00'> can </font></b> be run in a Colab GPU session (go to page menu: `Runtime`->  `Change runtime type` -> select `GPU` and confirm
- Cells named as  <b><font color='#56b4e9'>PRELIMINARY OPERATIONS </font></b> have to be run <b><font color='#d55c00'>ONCE only at the start</font></b>  and  skipped for new runnings.
- <b><font color='#d55c00'>ONE</font></b> single sequence at the time can be processed by the pipeline. 
- A  <b><font color='#d55c00'>new run</font></b> can be perform input direcly the new structure in the pdb upload cell and run the pipeline cells again

****

## <b><font color='#009e74'>PRELIMINARY OPERATIONS </font></b>
These cells should be run once at the start of the notebook
****

In [1]:
#@title Install condacolab
#@markdown Run this cell to install condacolab. After running this cell the kernel will be automatically restarted (wait ~1min before run the next cell)

#@markdown **N.B: This cell should be run only ONCE at the START of the notebook.**
try:
    import google.colab
    !pip install condacolab
    import condacolab
    condacolab.install()
except ModuleNotFoundError:
    pass

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:14
🔁 Restarting kernel...


In [2]:
#@title Update gcc
#@markdown Run this cell to install gcc-9. This is required for GROMACS
!sudo echo 'deb [arch=amd64] http://archive.ubuntu.com/ubuntu focal main universe' >> /etc/apt/sources.list
!sudo apt update
!sudo apt install -y gcc-9 g++-9
!ln -snf /usr/bin/gcc-9 /usr/bin/gcc
!ln -snf /usr/bin/g++-9 /usr/bin/g++

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,013 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:14 http://ppa.launchpad

In [2]:
#@title Installing biopython
!pip install biopython
from Bio.PDB import *
import Bio.SeqUtils
import Bio.SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 55.1 MB/s eta 0:00:00


In [3]:
#@title Installing PDBfixer
#@markdown The repair of PDB is now done with PDBfixer

#@markdown We can also add other options, like modeler, later on
!conda install -q -y -c conda-forge pdbfixer openmm

Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - openmm
    - pdbfixer


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ocl-icd-2.3.1              |       h7f98852_0         119 KB  conda-forge
    ocl-icd-system-1.0.0       |                1           4 KB  conda-forge
    openmm-8.0.0               |   py38he9472fe_0        10.7 MB  conda-forge
    pdbfixer-1.8.1             |     pyh6c4a22f_0         498 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        11.3 MB

The following NEW packages will be INSTALLED:

  ocl-icd            conda-forge/linux-64::ocl-icd-2.3.1-h7f98852_0 
  ocl-icd-system     conda-forge/linux-64::ocl-icd-system-1.0.0-1 
  openmm             conda-forge/linux-64::openmm-8.0.0-py38he9472fe_0 
  pdbf

In [4]:
#@title Install py3Dmol and pdb-tools
!pip install py3Dmol
import py3Dmol
!pip install pdb-tools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 4.4 MB/s eta 0:00:00


In [10]:
#@title python libs
!pip install wget
import os
import subprocess
import shutil
import pdbfixer
from openmm.app import PDBFile
import wget
import tarfile
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from google.colab import files

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=f2541d248e9012b224822179ab604253c2add2baf94be252f700fe289189a658
  Stored in directory: /root/.cache/pip/wheels/46/4a/43/6e71c9584e8b20b326931deaf89f11bf4db1bda170ac38e7b4
Successfully built wget


In [5]:
#@title Install Torch and TorchAni
!conda install pytorch torchvision -c pytorch
!conda install -c conda-forge torchani=2.2.0 cudatoolkit
!conda install scikit-learn

Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - pytorch
    - torchvision


The follow

In [6]:
#@title Install ASE and Joblib
!conda install -c conda-forge ase
!conda install -c anaconda joblib

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done

# All requested packages already installed.

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - joblib


The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2022.12.~ --> anaconda::ca-certificates-2023.01.10-h06a4308_0 

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi            conda-fo

In [7]:
#@title Get pKa-ANI
!git clone https://github.com/isayevlab/pKa-ANI.git

Cloning into 'pKa-ANI'...
remote: Enumerating objects: 206, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 206 (delta 80), reused 78 (delta 42), pack-reused 69
Receiving objects: 100% (206/206), 39.70 MiB | 13.18 MiB/s, done.
Resolving deltas: 100% (108/108), done.


In [8]:
#@title Install pKa-ANI
%%bash
cd pKa-ANI
python setup.py install

running install
running bdist_egg
running egg_info
creating pkaani.egg-info
writing pkaani.egg-info/PKG-INFO
writing dependency_links to pkaani.egg-info/dependency_links.txt
writing entry points to pkaani.egg-info/entry_points.txt
writing top-level names to pkaani.egg-info/top_level.txt
writing manifest file 'pkaani.egg-info/SOURCES.txt'
reading manifest file 'pkaani.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE'
writing manifest file 'pkaani.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/pkaani
copying pkaani/pkaani.py -> build/lib/pkaani
copying pkaani/__init__.py -> build/lib/pkaani
copying pkaani/prep_pdb.py -> build/lib/pkaani
copying pkaani/ani_descriptors.py -> build/lib/pkaani
copying pkaani/ase_io_proteindatabank_mod.py -> build/lib/pkaani
copying pkaani/run.py -> build/lib/pkaani
copying pkaani/__main__.py -> bui

/usr/local/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/setuptools/command/build_py.py:202: SetuptoolsDeprecationWarning:     Installing 'pkaani.models' as data is deprecated, please list it in `packages`.
    !!


    ############################
    # Package would be ignored #
    ############################
    Python recognizes 'pkaani.models' as an importable package,
    but it is not listed in the `packages` configuration of setuptools.

    'pkaani.models' has been automatically added to the distribution only
    because it may contain data files, but this behavior is likely to chang

In [11]:
#@title Install Modeller
#@markdown You will need to obtain modeller Licence Key from 
#@markdown https://salilab.org/modeller/. 
modellerkey = 'MODELIRANJE' #@param {type:"string"}
os.environ['KEY_MODELLER']=modellerkey
!conda config --add channels salilab
!conda install modeller
from modeller import *
from modeller.automodel import *

Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - modeller


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    hdf5-1107-1.10.7           |                0         3.1 MB  salilab
    modeller-10.4              |   py38h9bf148f_0        16.8 MB  salilab
    zlib-1.2.13                |       h166bdaf_4          92 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        19.9 MB

The following NEW packages will be INSTALLED:

  hdf5-1107          salilab/linux-64::hdf5-1107-1.10.7-0 
  modeller           salilab/linux-64::modeller-10.4-py38h9bf148f_0 
  zlib     

In [12]:
#@title Install MDAnalysis
!pip install --upgrade MDAnalysis
import MDAnalysis as mda
from MDAnalysis.analysis import rms, align

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.2/408.2 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 36.2 MB/s eta 0:00:00


# Set up parameters

In [13]:
#@title Gromacs setup
#@markdown We expect GROMACS to be preinstalled with one of our scripts and
#@markdown stored on Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

#@markdown Select GROMACS version from dropdown menu
gmx_version = '2022' #@param ["2021","2022"]
#@markdown - Currently versions 2021(2021.5) and 2022(2022.3) are availablle
gdrive_path = str('/content/gdrive/MyDrive/GROMACS/GROMACS/gromacs-'+gmx_version+'.zip')
local_path = str('/content/gromacs-'+gmx_version)
os.environ['GMX_LOCAL_PATH']=local_path
os.environ['GMX_ZIP_PATH']=gdrive_path

##@title Extract GROMACS binaries
!unzip $GMX_ZIP_PATH

Mounted at /content/gdrive
Archive:  /content/gdrive/MyDrive/GROMACS/GROMACS/gromacs-2022.zip
   creating: gromacs-2022/
   creating: gromacs-2022/lib/
  inflating: gromacs-2022/lib/libgmxapi.so.0.3.1  
  inflating: gromacs-2022/lib/libgmxapi.so  
  inflating: gromacs-2022/lib/libmuparser.so.2.3.2  
  inflating: gromacs-2022/lib/libgmxapi.so.0  
  inflating: gromacs-2022/lib/libnblib_gmx.so.0.1.0  
  inflating: gromacs-2022/lib/libgromacs.so  
  inflating: gromacs-2022/lib/libgromacs.so.7  
  inflating: gromacs-2022/lib/libgromacs.so.7.0.0  
   creating: gromacs-2022/lib/pkgconfig/
  inflating: gromacs-2022/lib/pkgconfig/muparser.pc  
  inflating: gromacs-2022/lib/pkgconfig/libgromacs.pc  
  inflating: gromacs-2022/lib/libnblib_gmx.so  
  inflating: gromacs-2022/lib/libmuparser.so.2  
  inflating: gromacs-2022/lib/libmuparser.so  
  inflating: gromacs-2022/lib/libnblib_gmx.so.0  
   creating: gromacs-2022/include/
  inflating: gromacs-2022/include/muParserToken.h  
  inflating: gromacs

In [14]:
#@title Test GROMACS installation
%%bash
chmod -R 755 $GMX_LOCAL_PATH
source $GMX_LOCAL_PATH/bin/GMXRC
gmx grompp -h

SYNOPSIS

gmx grompp [-f [<.mdp>]] [-c [<.gro/.g96/...>]] [-r [<.gro/.g96/...>]]
           [-rb [<.gro/.g96/...>]] [-n [<.ndx>]] [-p [<.top>]]
           [-t [<.trr/.cpt/...>]] [-e [<.edr>]] [-qmi [<.inp>]]
           [-ref [<.trr/.cpt/...>]] [-po [<.mdp>]] [-pp [<.top>]]
           [-o [<.tpr>]] [-imd [<.gro>]] [-[no]v] [-time <real>]
           [-[no]rmvsbds] [-maxwarn <int>] [-[no]zero] [-[no]renum]

DESCRIPTION

gmx grompp (the gromacs preprocessor) reads a molecular topology file, checks
the validity of the file, expands the topology from a molecular description to
an atomic description. The topology file contains information about molecule
types and the number of molecules, the preprocessor copies each molecule as
needed. There is no limitation on the number of molecule types. Bonds and
bond-angles can be converted into constraints, separately for hydrogens and
heavy atoms. Then a coordinate file is read and velocities can be generated
from a Maxwellian distribution if requested

                      :-) GROMACS - gmx grompp, 2022.3 (-:

Executable:   /content/gromacs-2022/bin/gmx
Data prefix:  /content/gromacs-2022
Working dir:  /content
Command line:
  gmx grompp -h


GROMACS reminds you: "In mathematics you don't understand things, you just get used to them" (John von Neumann)



In [44]:
#PP this is work around the strange bug related to the encoding
# It should be UTF-8, but sometimes it becomes ANSI and nothing work
import locale
print(locale.getpreferredencoding())
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
print(locale.getpreferredencoding())

UTF-8
UTF-8


In [84]:
#@title <b><font color='#56b4e9'> PDB Input</font></b>

#@markdown Choose between <b><font color='#d55c00'> ONE</font></b> of the possible input sources for the target pdb and <ins>leave the other cells empty or unmarked</ins>
#@markdown - AlphaFold2 PDB via Uniprot ID:
uniprot_id =''#@param {type:"string"}
#@markdown - PDB ID (imported from RCSB PDB):
pdb_id =''#@param {type:"string"}
#@markdown - Upload PDB from local drive (in this case you will be prompt to the file upload later)
pdb_local =False#@param {type:"boolean"}
#@markdown - Use PDB from google drive (insert path to pdb file in the google drive)
# '/content/gdrive/MyDrive/NESP/data/targetA_relaxed.pdb'

#pdb_gdrive ='/content/gdrive/MyDrive/NESP/data/targetA_relaxed.pdb'#@param {type:"string"}
pdb_gdrive = ''#@param {type:"string"}

# sequence to test: VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK

#@markdown - Insert protein sequence to model the target structure using ESMFold
run_esmfold = 'VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK'#@param {type:"string"}


#@markdown - Insert protein sequence to model the target structure using AlphaFold
run_alphafold =''#@param {type:"string"}

#@markdown

#@markdown Select target chain (default is A)
chain_id='A' #@param {type:'string'}

project_dir = "/content/alphagmx_project/"
pdb_path = os.path.join(project_dir, 'target.pdb')
pdb_preprocessed_path = os.path.join(project_dir, 'target_preprocessed.pdb')

if os.path.exists(project_dir):
    print(f'Warning! {project_dir} exists, all files in this directory will be deleted')
    #os.remove(project_dir)
    shutil.rmtree(project_dir) 
    os.mkdir(project_dir)
else:
    os.mkdir(project_dir)

if (uniprot_id !='') and (len(uniprot_id)==6) : 
    print('Loading PDB by Uniprot_ID')
    subprocess.call(['curl','-s','-f',f'https://alphafold.ebi.ac.uk/files/AF-{uniprot_id}-F1-model_v2.pdb','-o', pdb_path])
elif (pdb_id !='') and (len(pdb_id)==4):
    print('Loading PDB by PDB_ID')
    subprocess.call(['curl','-s','-f',f'https://files.rcsb.org/download/{pdb_id}.pdb','-o', pdb_path])
elif pdb_local:
    print('Upload PDB file:')
    uploaded_pdb = files.upload()
    for fn in uploaded_pdb.keys():
        shutil.copy(fn, pdb_path)
elif pdb_gdrive!='' and os.path.exists(pdb_gdrive):
    print('Using PDB file from gdrive:')
    shutil.copy(pdb_gdrive, pdb_path)
elif run_esmfold!='':
    sequence = run_esmfold
    print('Using ESMFold API to model the structure')
    os.system(f'curl -X POST --data "{sequence}" https://api.esmatlas.com/foldSequence/v1/pdb/ > {pdb_path}')
elif run_alphafold!='':
    sequence = run_alphafold
    print('RUNNING ALPHAFOLD TODO')
else:
    print(f'ERROR: unrecognized pdb input')


#@markdown N.B. This cell will also perform preliminary operations to correcly format the uploaded PDB

#@markdown - mark to preprocess pdb (keep only single chain, delete hetatms)
flag_preprocess =True#@param {type:"boolean"}

if flag_preprocess:
    print('Preprocessing PDBs...')
    #!pdb_selchain -"$chain_id" "$pdb_path" | pdb_delhetatm | pdb_delres --999:0:1 | pdb_fixinsert | pdb_tidy  > "$pdb_preprocessed_path"
    !pdb_selchain -"$chain_id" "$pdb_path" | pdb_delhetatm | pdb_fixinsert | pdb_tidy  > "$pdb_preprocessed_path"
    print(f'Kept only chain {chain_id}, deleted hetatms, fixed insertion code, sorted residue ids, formatted pdb file...')
    parser = PDBParser()
    io=PDBIO()
    structure = parser.get_structure('X', pdb_preprocessed_path)

    #And here we set the residue conformation we want to keep
    keepAltID = "A"

    class KeepOneConfOnly(Select):  # Inherit methods from Select class
        def accept_atom(self, atom):
            if (not atom.is_disordered()) or atom.get_altloc() == keepAltID:
                atom.set_altloc(" ")  # Eliminate alt location ID before output.
                return True
            else:  # Alt location was not one to be output.
                return False
            # end of accept_atom()

    #This will keep only conformation for each residue
    io.set_structure(structure)

    io.save(os.path.join(pdb_preprocessed_path), select=KeepOneConfOnly())
    print('Removed alternative side chain conformations...')

else:
    pdb_preprocessed_path = pdb_path

if os.path.exists(pdb_preprocessed_path):
  print(f"Pre-processing PDBs correctly ended")
else:
  print(f"Pre-processing PDB didn't end correcly, please check the inputs")

#@markdown ****

Warning! /content/alphagmx_project/ exists, all files in this directory will be deleted
Using ESMFold API to model the structure
Preprocessing PDBs...
Kept only chain A, deleted hetatms, fixed insertion code, sorted residue ids, formatted pdb file...
Removed alternative side chain conformations...
Pre-processing PDBs correctly ended


In [86]:
#@title Fix missing residues
#@markdown Select the software to deal with missing residues. Currently, only
#@markdown `PDBFixer` is available.
missing_residues_soft = 'pdbfixer' #@param ["pdbfixer","modeller","-"]

In [87]:
#@title Run pdbfixer

def prepare_protein(pdb_file, ignore_missing_residues=False, ignore_terminal_missing_residues=True, pdb_out=None):
    """Use pdbfixer to prepare the protein from a PDB file. Hetero atoms such as ligands are 
    removed and non-standard residues replaced. Missing atoms to existing residues are added. 
    Missing residues are ignored by default, but can be included.
    
    Parameters
    ----------
    pdb_file: pathlib.Path or str
        PDB file containing the system to prepare.
    ignore_missing_residues: bool
        If missing residues should be ignored or built.
    ignore_terminal_missing_residues: bool
        If missing residues at the beginning and the end of a chain should be ignored or built.
    pdb_out: pathlib.Path or str
        Output PDB file containing the system to simulate.
    
    Returns
    -------
    fixer: pdbfixer.pdbfixer.PDBFixer
        Prepared protein system.
    """
    fixer = pdbfixer.PDBFixer(str(pdb_file))
    fixer.removeHeterogens()  # co-crystallized ligands are unknown to PDBFixer
    fixer.findMissingResidues()  # identify missing residues, needed for identification of missing atoms
    
    # if missing terminal residues shall be ignored, remove them from the dictionary
    if ignore_terminal_missing_residues:  
        chains = list(fixer.topology.chains())
        keys = fixer.missingResidues.keys()
        for key in list(keys):
            chain = chains[key[0]]
            if key[1] == 0 or key[1] == len(list(chain.residues())):
                del fixer.missingResidues[key]
                
    # if all missing residues shall be ignored ignored, clear the dictionary
    if ignore_missing_residues:  
        fixer.missingResidues = {}
        
    fixer.findNonstandardResidues()  # find non-standard residue
    fixer.replaceNonstandardResidues()  # replace non-standard residues with standard one
    fixer.findMissingAtoms()  # find missing heavy atoms
    fixer.addMissingAtoms()  # add missing atoms
    PDBFile.writeFile(fixer.topology, fixer.positions, open(pdb_out, 'w'))

if missing_residues_soft == '-':
    pass
elif missing_residues_soft == "pdbfixer":
    prepare_protein(pdb_preprocessed_path, pdb_out=pdb_preprocessed_path)
else:
    print('Warning! Currently only pdbfixer option is available for missing residues')

u = mda.Universe(pdb_preprocessed_path)
#u.select_atoms("not resname HOH").write(os.path.splitext(pdb_preprocessed_path)[0]+"_no_water.pdb")
u.select_atoms("not resname HOH").write(os.path.splitext(pdb_preprocessed_path)[0]+".pdb") #PP: why do we need a separate file for _no_water.pdb?

/usr/local/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:753: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/usr/local/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


# Display structure

In [88]:
def show_pdb(pdb_file, show_sidechains=False, show_mainchains=False):
  model_name = "Parsed structure"
  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
  view.addModel(open(pdb_file,'r').read(),'pdb')

  view.setStyle({'cartoon': {'color':'spectrum'}})
  
  if show_sidechains:
    BB = ['C','O','N']
    view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]},
                        {'sphere':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})  
  if show_mainchains:
    BB = ['C','O','N','CA']
    view.addStyle({'atom':BB},{'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})

  view.zoomTo()
  return view

In [89]:
show_pdb(pdb_preprocessed_path)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Add mutants

In [51]:
#@title Mutants to simulate
#@markdown If you want to simulate mutants upload a file with a list of mutants
#@markdown The file should have the following format:

#@markdown Commented lines start with \#.
#@markdown Each line corresponds to a single mutant. Each mutation is described
#@markdown as XNY, where X is one-letter code of the original amino acid, N
#@markdown is the residue id of the mutated amino acid in the PDB file, 
#@markdown and Y is the mutated amino acid.

simulate_mutants = True #@param {type:"boolean"}
if simulate_mutants:
    uploaded = files.upload()
    mutations_file = list(uploaded.keys())[-1]

Saving mutations.txt to mutations (1).txt


In [91]:
#@title Read mutation file and prepare alignments
alphabet = list(Bio.SeqUtils.IUPACData.protein_letters_1to3.keys())
alphabet.append("-")

parser = PDBParser()
structure = parser.get_structure('X', pdb_preprocessed_path)

residues = ""
resids = []
for model in structure:
    for chain in model:
        for i, residue in enumerate(chain.get_residues()):
            residue_1l = Bio.SeqUtils.seq1(residue.resname)
            resid = residue.id[1]
            if residue_1l != "X":
                residues += residue_1l
                resids.append(resid)

record = SeqRecord(Seq(residues), id="0", name="template", 
                   description="input protein")

mutant_n = 0
with open(mutations_file, "r") as f:
    for line in f:
        if line.startswith("#") or line.strip() == "":
            continue

        folder_mask = project_dir+"/md_mutant_"

        if not os.path.exists(f"{folder_mask}{mutant_n}"):
            os.mkdir(f"{folder_mask}{mutant_n}")
        
        new_sequence = residues
        for mutation in line.split():
            original_AA = mutation[0].upper()
            if original_AA not in alphabet:
                raise Exception("The wrong amino acid code is provided"+\
                                f"in the mutation file for mutation {mutation}")
            
            mutant_AA = mutation[-1].upper()
            if mutant_AA not in alphabet:
                raise Exception("The wrong amino acid code is provided for in"+\
                                f"in the mutation file for mutation {mutation}")
                
            try:
                resid = int(mutation[1:-1])
            except:
                print(f"Can't get resid for mutation {mutation}. Check you input")
                raise
            
            if residues[resids.index(resid)] != original_AA:
                raise Exception(f"In the PDB file resid {resid} has amino acid"+\
                                f" {residues[resids.index(resid)]} while in"+\
                                f"mutation file {original_AA} is provide")
            new_sequence = list(new_sequence)
            new_sequence[resids.index(resid)] = mutant_AA
            new_sequence = "".join(new_sequence)

        record_mutant = SeqRecord(Seq(new_sequence), id="1", name="mutant", 
                description="mutated protein")
        
        handle = open(f"{folder_mask}{mutant_n}/alignment.ali", "w")
        writer = Bio.SeqIO.PirIO.PirWriter(handle)
        writer.write_record(record)
        writer.write_record(record_mutant)
        handle.close()
        mutant_n += 1

# setting environ variables for modeller 
os.environ['N_MUTANT'] = str(mutant_n)
os.environ['N_RESID'] = str(len(residues))
os.environ['PDB'] = os.path.basename(pdb_preprocessed_path).split('.')[0]
if not os.path.exists(os.path.join(project_dir, 'md_base')):
    os.mkdir(os.path.join(project_dir, 'md_base'))
os.environ['MD_PATH'] = os.path.join(project_dir, 'md_base')
os.environ['ALPHAGMX'] = project_dir
#fn_pdb_md = os.path.join(os.path.dirname(pdb_preprocessed_path), 'md_base', os.path.splitext(os.path.basename(pdb_preprocessed_path))[0]+"_no_water.pdb")
fn_pdb_md = os.path.join(os.path.dirname(pdb_preprocessed_path), 'md_base', os.path.splitext(os.path.basename(pdb_preprocessed_path))[0]+".pdb")
#shutil.copy(os.path.splitext(pdb_preprocessed_path)[0]+"_no_water.pdb", fn_pdb_md)
shutil.copy(os.path.splitext(pdb_preprocessed_path)[0]+".pdb", fn_pdb_md)
os.environ['PDB4MD'] = fn_pdb_md.split('.')[0]

In [95]:
!head -n 20 alphagmx_project/md_mutant_0/alignment.ali

>P1;0
structureX:/content/alphagmx_project/md_base/target_preprocessed:1:A:221:A::::
VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQ
RVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGT
NAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKAL
GSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK*
>P1;1
sequence:mutated protein:1::221:::::
APVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQ
RVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGT
NAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKAL
GSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK*


In [94]:
#@title Correct naming for alignment file
%%bash
#echo "s/template - input protein/structureX:${PDB4MD}:1:A:${N_RESID}:A::::/g"
#echo "s/template - input protein/structureX:\/content\/alphagmx\/md_base\/${PDB}_no_water:1:A:${N_RESID}:A::::/g"
for (( i=0; i<${N_MUTANT}; i++ ))
do
sed -i 's/XX/P1/g' alphagmx_project/md_mutant_${i}/alignment.ali
#sed -i "s/template - input protein/structureX:\/content\/alphagmx\/md_base\/${PDB}_no_water:1:A:${N_RESID}:A::::/g" alphagmx/md_mutant_${i}/alignment.ali
#sed -i "s/template - input protein/structureX:${PDB4MD}:1:A:${N_RESID}:A::::/g" alphagmx/md_mutant_${i}/alignment.ali
#sed -i "s/template - input protein/structureX:\/content\/alphagmx\/md_base\/target_preprocessed_no_water:1:A:${N_RESID}:A::::/g" alphagmx/md_mutant_${i}/alignment.ali
sed -i "s/template - input protein/structureX:\/content\/alphagmx_project\/md_base\/target_preprocessed:1:A:${N_RESID}:A::::/g" alphagmx_project/md_mutant_${i}/alignment.ali
sed -i "s/mutant - mutated protein/sequence:mutated protein:1::${N_RESID}:::::/g" alphagmx_project/md_mutant_${i}/alignment.ali
done
#TODO: A is for chain id? Is it always A here?
#TODO: this cell will be completed even if something goes wrong

In [96]:
#@title Run modeller
for i in range(mutant_n):
    os.chdir(project_dir+f"/md_mutant_{i}")
    env = Environ()
    a = AutoModel(env, alnfile='alignment.ali',
                  knowns='0', sequence='1',
                  assess_methods=(assess.DOPE,
                                  #soap_protein_od.Scorer(),
                                  assess.GA341))
    a.starting_model = 1
    a.ending_model = 1
    a.make()
    for file in os.listdir("."):
        if file.endswith("pdb"):
            shutil.copy2(file, 'input.pdb')
os.chdir("/content")


check_ali___> Checking the sequence-structure alignment. 

Implied intrachain target CA(i)-CA(i+1) distances longer than  8.0 angstroms:

ALN_POS  TMPL  RID1  RID2  NAM1  NAM2     DIST
----------------------------------------------
END OF TABLE
read_to_681_> topology.submodel read from topology file:        3
patch_s_522_> Number of disulfides patched in MODEL:        2
mdtrsr__446W> A potential that relies on one protein is used, yet you have at
              least one known structure available. MDT, not library, potential is used.
0 atoms in HETATM/BLK residues constrained
to protein atoms within 2.30 angstroms
and protein CA atoms within 10.00 angstroms
0 atoms in residues without defined topology
constrained to be rigid bodies
condens_443_> Restraints marked for deletion were removed.
              Total number of restraints before, now:    23449    21841
iupac_m_397W> Atoms were not swapped because of the uncertainty of how to handle the H atom.
>> Model assessment by DOPE potent

# Calculate most probable protonation states 

In [97]:
#@ Run pkaAni
%%bash
cd ${MD_PATH}
in_pdb=${PDB4MD}.pdb
pkaani -i $in_pdb -p True
for (( i=0; i<${N_MUTANT}; i++ ))
do
cd ../md_mutant_${i}
in_pdb=input.pdb
pkaani -i $in_pdb -p True
done

Preparing /content/alphagmx_project/md_base/target_preprocessed for pKa calculations
Loading pKa-ANI Models and ANI-2x...
Finished Loading.
Calculating pKa for /content/alphagmx_project/md_base/target_preprocessed.pdb
Preparing input for pKa calculations
Loading pKa-ANI Models and ANI-2x...
Finished Loading.
Calculating pKa for input.pdb


/usr/local/lib/python3.8/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.8/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.8/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at you

In [99]:
#@title Select the force field
#@markdown We currently provide access to three force fields: AMBER99-ILDN, CHARMM36m, and AMBER14SB
ff = 'charmm36' #@param {type:"string"} ["amber99", "amber14", "charmm36"]
os.environ['FF']=ff

In [100]:
#@title Get force fields for pdb2gmx
if ff == 'amber14':
    wget.download("https://github.com/pbuslaev/alphagmx/raw/main/forcefields/amber14sb.ff.tar.gz")
    tar = tarfile.open("amber14sb.ff.tar.gz")
    tar.extractall()
    tar.close()
if ff == 'charmm36':
    wget.download("http://mackerell.umaryland.edu/download.php?filename=CHARMM_ff_params_files/charmm36-jul2021.ff.tgz")
    tar = tarfile.open("charmm36-jul2021.ff.tgz")
    tar.extractall()
    tar.close()


In [101]:
#@title Change residue names in accordance with pKa calculation
pH = 7
names              = ["ASP",  "GLU",  "HIS", "LYS"]
charmmProtonated   = ["ASPP", "GLUP", "HSP", "LYS"]
charmmDeprotonated = ["ASP",  "GLU",  "HIS", "LSN"]
amberProtonated    = ["ASH",  "GLH",  "HIP", "LYS"]
amberDeprotonated  = ["ASP",  "GLU",  "HIS", "LYN"]
if ff == 'charmm36':
    protNames   = charmmProtonated
    deprotNames = charmmDeprotonated
else:
    protNames   = amberProtonated
    deprotNames = amberDeprotonated
for i in range(-1, mutant_n, 1):
    parser = PDBParser()
    if i == -1:
        #structure = parser.get_structure('X', path+"pdb"+pdbid+".ent") #PP: why .ent here?
        structure = parser.get_structure('X', pdb_preprocessed_path)
        #logname = path+pdbid+"_processed_pka.log"
        logname =  os.environ['PDB4MD']+"_pka.log"
        #outpath = path+"input_pka.pdb"
        outpath = os.path.join(os.environ['MD_PATH'], "input_pka.pdb")
    else:
        structure = parser.get_structure('X', os.path.join(os.environ["ALPHAGMX"], f"md_mutant_{i}/input.pdb"))
        logname = os.path.join(os.environ["ALPHAGMX"], f"md_mutant_{i}/input_pka.log")
        outpath = os.path.join(os.environ["ALPHAGMX"], f"md_mutant_{i}/input_pka.pdb")
    residues = []
    pkas     = []
    with open(logname, "r") as f:
        for line in f:
            if line.startswith("Residue"):
                continue
            if line.strip() == "":
                continue
            resname, resid, chain, pka = line.split()[0].split("-")[0], \
                                         int(line.split()[0].split("-")[1]), \
                                         line.split()[1], \
                                         float(line.split()[2])
            residues.append((resname, resid, chain))
            pkas.append(pka)

    for model in structure:
        for chain in model:
            for i, residue in enumerate(chain.get_residues()):
                if (residue.resname, residue.id[1], chain.id) in residues:
                    id = residues.index((residue.resname, residue.id[1], chain.id))
                    if (residue.resname in names) and pH < pkas[id]:
                        residue.resname = \
                            protNames[names.index(residue.resname)]
                    if (residue.resname in names) and pH > pkas[id]:
                        residue.resname = \
                            deprotNames[names.index(residue.resname)]

    pdb_io = PDBIO()
    pdb_io.set_structure(structure)
    pdb_io.save(outpath)             

In [79]:
# #PP: this is can be done in the next cell within the cycle
# %%bash
# # Declare a filename array
# arrVar=(${MD_PATH})

# # Add mutants
# for (( i=0; i<${N_MUTANT}; i++ ))
# do
# #arrVar+=("./md_mutant_${i}")
# arrVar+=(${ALPHAGMX}"/md_mutant_${i}")
# done

# #echo "${arrVar[@]}"

# for value in "${arrVar[@]}"
# do

# sed -i 's/GLUP /GLUP/g' ${value}/input_pka.pdb
# sed -i 's/ASPP /ASPP/g' ${value}/input_pka.pdb

# done

/content/alphagmx/md_base /content/alphagmx//md_mutant_0


In [102]:
#@title Run pdb2gmx, editconf, solvate and genion
%%bash
git clone https://github.com/pbuslaev/alphagmx.git

source $GMX_LOCAL_PATH/bin/GMXRC
# Declare a filename array
arrVar=(${MD_PATH})

# Add mutants
for (( i=0; i<${N_MUTANT}; i++ ))
do
#arrVar+=("./md_mutant_${i}")
arrVar+=(${ALPHAGMX}"/md_mutant_${i}")
done

#echo "${arrVar[@]}"

for value in "${arrVar[@]}"
do

echo ${value}
sed -i 's/GLUP /GLUP/g' ${value}/input_pka.pdb
sed -i 's/ASPP /ASPP/g' ${value}/input_pka.pdb

cd /content/
in_pdb=input_pka.pdb
if [ ${FF} == "amber14" ]; then
cp -r amber14sb.ff $value/.
cp alphagmx/parameters/amber/*mdp $value/.
cd $value
gmx pdb2gmx -f $in_pdb -o 1_prot.gro -p -ignh -ter -water spce -ff amber14sb
fi
if [ ${FF} == "amber99" ]; then
cp alphagmx/parameters/amber/*mdp $value/.
cd $value
gmx pdb2gmx -f $in_pdb -o 1_prot.gro -p -ignh -ter -water spce -ff amber99sb-ildn
fi
if [ ${FF} == "charmm36" ]; then
cp -r charmm36-jul2021.ff $value/.
cp alphagmx/parameters/charmm/*mdp $value/.
cd $value
gmx pdb2gmx -f $in_pdb -o 1_prot.gro -p -ignh -water tip3p -ff charmm36-jul2021
fi
gmx editconf -f 1_prot.gro -o 2_box.gro -d 2
gmx solvate -cp 2_box.gro -cs -p topol.top -o 3_solv.gro
# Here we should apply some tricks Issue !4
# https://github.com/pbuslaev/alphagmx/issues/4
gmx grompp -f ions.mdp -c 3_solv.gro -p topol.top -o 4_ions.tpr -maxwarn 1
#This is a trick to provide interactive options to gmx
echo "SOL" > options
echo " " >> options
if [ ${FF} == "charmm36" ]; then
gmx genion -s 4_ions.tpr -p topol.top -o 4_ions.gro -neutral -conc 0.15 -pname SOD -nname CLA < options
fi
if [ ${FF} == "amber99" ]; then
gmx genion -s 4_ions.tpr -p topol.top -o 4_ions.gro -neutral -conc 0.15 -pname NA -nname CL < options
fi
if [ ${FF} == "amber14" ]; then
gmx genion -s 4_ions.tpr -p topol.top -o 4_ions.gro -neutral -conc 0.15 -pname NA -nname CL < options
fi
done

/content/alphagmx_project/md_base
Using the Charmm36-jul2021 force field in directory ./charmm36-jul2021.ff

going to rename ./charmm36-jul2021.ff/aminoacids.r2b

going to rename ./charmm36-jul2021.ff/carb.r2b

going to rename ./charmm36-jul2021.ff/cgenff.r2b

going to rename ./charmm36-jul2021.ff/ethers.r2b

going to rename ./charmm36-jul2021.ff/lipid.r2b

going to rename ./charmm36-jul2021.ff/metals.r2b

going to rename ./charmm36-jul2021.ff/na.r2b

going to rename ./charmm36-jul2021.ff/silicates.r2b

going to rename ./charmm36-jul2021.ff/solvent.r2b
Reading input_pka.pdb...
Read '', 1688 atoms

Analyzing pdb file
Splitting chemical chains based on TER records or chain id changing.

There are 1 chains and 0 blocks of water and 221 residues with 1688 atoms

  chain  #res #atoms

  1 'A'   221   1688  

All occupancies are one

Reading residue database... (Charmm36-jul2021)

Processing chain 1 'A' (1688 atoms, 221 residues)

Identified residue VAL1 as a starting terminus.

Identified r

Cloning into 'alphagmx'...
                     :-) GROMACS - gmx pdb2gmx, 2022.3 (-:

Executable:   /content/gromacs-2022/bin/gmx
Data prefix:  /content/gromacs-2022
Working dir:  /content/alphagmx_project/md_base
Command line:
  gmx pdb2gmx -f input_pka.pdb -o 1_prot.gro -p -ignh -water tip3p -ff charmm36-jul2021

Opening force field file ./charmm36-jul2021.ff/aminoacids.r2b
Opening force field file ./charmm36-jul2021.ff/carb.r2b
Opening force field file ./charmm36-jul2021.ff/cgenff.r2b
Opening force field file ./charmm36-jul2021.ff/ethers.r2b
Opening force field file ./charmm36-jul2021.ff/lipid.r2b
Opening force field file ./charmm36-jul2021.ff/metals.r2b
Opening force field file ./charmm36-jul2021.ff/na.r2b
Opening force field file ./charmm36-jul2021.ff/silicates.r2b
Opening force field file ./charmm36-jul2021.ff/solvent.r2b
All occupancies are one
Opening force field file ./charmm36-jul2021.ff/atomtypes.atp
Opening force field file ./charmm36-jul2021.ff/aminoacids.rtp
Opening forc

In [103]:
#@title Run equilibration
%%bash
source $GMX_LOCAL_PATH/bin/GMXRC
# Declare a filename array
arrVar=(${MD_PATH})

# Add mutants
for (( i=0; i<${N_MUTANT}; i++ ))
do
#arrVar+=("./md_mutant_${i}")
arrVar+=(${ALPHAGMX}"/md_mutant_${i}")
done

for value in "${arrVar[@]}"
do
cd /content/
cd $value
gmx grompp -f min.mdp -c 4_ions.gro -p topol.top -o 5_min.tpr -r 4_ions.gro 
gmx mdrun -deffnm 5_min
gmx grompp -f nvt.mdp -c 5_min.gro -p topol.top -o 6_nvt.tpr -r 5_min.gro 
gmx mdrun -deffnm 6_nvt
gmx grompp -f npt.mdp -c 6_nvt.gro -p topol.top -o 7_npt.tpr -r 6_nvt.gro 
gmx mdrun -deffnm 7_npt
done

Setting the LD random seed to -33555497

Generated 167799 of the 167910 non-bonded parameter combinations

Generated 117519 of the 167910 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_A'

Excluding 2 bonded neighbours molecule type 'SOL'

Excluding 3 bonded neighbours molecule type 'SOD'

Excluding 3 bonded neighbours molecule type 'CLA'
Analysing residue names:
There are:   221    Protein residues
There are: 21790      Water residues
There are:   133      Other residues
Analysing Protein...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

The largest distance between excluded atoms is 0.453 nm
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 72x64x60, spacing 0.134 0.137 0.139

Estimate for the relative computational load of the PME mesh part: 0.15

This run will generate roughly 5 Mb of data
Setting the LD random seed to -641

Generated 167799 of the 167910 non-bonded parameter combi

                      :-) GROMACS - gmx grompp, 2022.3 (-:

Executable:   /content/gromacs-2022/bin/gmx
Data prefix:  /content/gromacs-2022
Working dir:  /content/alphagmx_project/md_base
Command line:
  gmx grompp -f min.mdp -c 4_ions.gro -p topol.top -o 5_min.tpr -r 4_ions.gro

Generating 1-4 interactions: fudge = 1
Number of degrees of freedom in T-Coupling group rest is 141087.00

GROMACS reminds you: "The Path Of the Righteous Man is Beset On All Sides With the Iniquities Of the Selfish and the Tyranny Of Evil Men." (Pulp Fiction)

                      :-) GROMACS - gmx mdrun, 2022.3 (-:

Executable:   /content/gromacs-2022/bin/gmx
Data prefix:  /content/gromacs-2022
Working dir:  /content/alphagmx_project/md_base
Command line:
  gmx mdrun -deffnm 5_min

Reading file 5_min.tpr, VERSION 2022.3 (single precision)
1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU


In [105]:
%%bash
source $GMX_LOCAL_PATH/bin/GMXRC
# Declare a filename array
arrVar=(${MD_PATH})

# Add mutants
for (( i=0; i<${N_MUTANT}; i++ ))
do
#arrVar+=("./md_mutant_${i}")
arrVar+=(${ALPHAGMX}"/md_mutant_${i}")
done

for value in "${arrVar[@]}"
do
cd /content/
cd $value
gmx grompp -f md.mdp -c 7_npt.gro -p topol.top -o run.tpr
gmx mdrun -deffnm run -nsteps 1000
done

Setting the LD random seed to -1449533441

Generated 167799 of the 167910 non-bonded parameter combinations

Generated 117519 of the 167910 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_A'

turning H bonds into constraints...

Excluding 2 bonded neighbours molecule type 'SOL'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'SOD'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'CLA'

turning H bonds into constraints...
Analysing residue names:
There are:   221    Protein residues
There are: 21790      Water residues
There are:   133      Other residues
Analysing Protein...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

The largest distance between excluded atoms is 0.414 nm

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K

Calculated rlist for 1x1 atom pair-list as 1.234 nm, buffer size 0.034 nm

Set rlist, assuming 

                      :-) GROMACS - gmx grompp, 2022.3 (-:

Executable:   /content/gromacs-2022/bin/gmx
Data prefix:  /content/gromacs-2022
Working dir:  /content/alphagmx_project/md_base
Command line:
  gmx grompp -f md.mdp -c 7_npt.gro -p topol.top -o run.tpr

Generating 1-4 interactions: fudge = 1
Number of degrees of freedom in T-Coupling group System is 139458.00

Back Off! I just backed up run.tpr to ./#run.tpr.1#

GROMACS reminds you: "And It Goes a Little Something Like This" (Tag Team)

                      :-) GROMACS - gmx mdrun, 2022.3 (-:

Executable:   /content/gromacs-2022/bin/gmx
Data prefix:  /content/gromacs-2022
Working dir:  /content/alphagmx_project/md_base
Command line:
  gmx mdrun -deffnm run -nsteps 10


Back Off! I just backed up run.log to ./#run.log.1#
Reading file run.tpr, VERSION 2022.3 (single precision)
Overriding nsteps with value passed on the command line: 10 steps, 0.02 ps
Changing nstlist from 10 to 100, rlist from 1.2 to 1.344


1 GPU selected for 

In [106]:
#@title make trajectories whole
%%bash
source $GMX_LOCAL_PATH/bin/GMXRC
# Declare a filename array
arrVar=(${MD_PATH})

# Add mutants
for (( i=0; i<${N_MUTANT}; i++ ))
do
#arrVar+=("./md_mutant_${i}")
arrVar+=(${ALPHAGMX}"/md_mutant_${i}")
done

for value in "${arrVar[@]}"
do
cd /content/
cd $value
echo 0 | gmx trjconv -f run.xtc -s run.tpr -pbc mol -o run_nj.xtc
done

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 0: 'System'
Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 0: 'System'


                     :-) GROMACS - gmx trjconv, 2022.3 (-:

Executable:   /content/gromacs-2022/bin/gmx
Data prefix:  /content/gromacs-2022
Working dir:  /content/alphagmx_project/md_base
Command line:
  gmx trjconv -f run.xtc -s run.tpr -pbc mol -o run_nj.xtc

Will write xtc: Compressed trajectory (portable xdr format): xtc
Reading file run.tpr, VERSION 2022.3 (single precision)
Reading file run.tpr, VERSION 2022.3 (single precision)
Group     0 (         System) has 68820 elements
Group     1 (        Protein) has  3317 elements
Group     2 (      Protein-H) has  1688 elements
Group     3 (        C-alpha) has   221 elements
Group     4 (       Backbone) has   663 elements
Group     5 (      MainChain) has   883 elements
Group     6 (   MainChain+Cb) has  1085 elements
Group     7 (    MainChain+H) has  1089 elements
Group     8 (      SideChain) has  2228 elements
Group     9 (    SideChain-H) has   805 elements
Group    10 (    Prot-Masses) has  3317 elements
Group    11 (    non-P

# Analysis

In [114]:
#@title RMSD
paths = [os.environ["MD_PATH"]]
for i in range(mutant_n):
    paths.append(os.path.join(project_dir, f"md_mutant_{i}/"))

first = True
rmsdList = []
for current_path in paths:
    mobile = mda.Universe(os.path.join(current_path, "run.gro"), os.path.join(current_path, "run_nj.xtc"))
    averageBB = align.AverageStructure(mobile, 
                                       mobile, 
                                       select='protein and backbone',
                                       ref_frame=0).run()
    refBB = averageBB.universe
    R_rmsd = rms.RMSD(mobile,  # universe to align
                     refBB,  # reference universe or atomgroup
                     select='protein and backbone')  # group to superimpose and calculate RMSD
        
    R_rmsd.run()
    if first:
        df_rmsd = pd.DataFrame(R_rmsd.results.rmsd,
                  columns=['Frame', 'Time (ns)',f'RMSD, {current_path}'])
        df_rmsd['Time (ns)'] /= 1000
        rmsdList.append(f'RMSD, {current_path}')
        first = False
    else:
        colname = f'RMSD, {current_path}'
        df_rmsd[colname] = R_rmsd.results.rmsd[:,2]
        rmsdList.append(colname)

df_rmsd_melt = df_rmsd.melt(id_vars='Time (ns)', value_vars=rmsdList)
df_rmsd_melt.rename(columns = {'value': 'RMSD (A)'}, inplace = True)
fig = px.line(df_rmsd_melt, x="Time (ns)", y="RMSD (A)",
              color='variable',
              line_shape="linear", render_mode="svg", 
              labels={ "Name CA": "RMSD(Å)" })
fig.show()

/usr/local/lib/python3.8/site-packages/MDAnalysis/analysis/align.py:952: DeprecationWarning:

The `universe` attribute was deprecated in MDAnalysis 2.0.0 and will be removed in MDAnalysis 3.0.0. Please use `results.universe` instead.



/content/alphagmx_project/md_base    Frame  Time (ns)  RMSD, /content/alphagmx_project/md_base
0    0.0        0.0                             6.289054e-07
/content/alphagmx_project/md_mutant_0/    Frame  Time (ns)  RMSD, /content/alphagmx_project/md_base  \
0    0.0        0.0                             6.289054e-07   

   RMSD, /content/alphagmx_project/md_mutant_0/  
0                                  3.630987e-07  


/usr/local/lib/python3.8/site-packages/MDAnalysis/analysis/align.py:952: DeprecationWarning:

The `universe` attribute was deprecated in MDAnalysis 2.0.0 and will be removed in MDAnalysis 3.0.0. Please use `results.universe` instead.

